# Building an AI Agent with LangGraph and AWS Bedrock

This notebook demonstrates how to build an AI agent from scratch using LangGraph and AWS Bedrock. We'll create an agent that can use tools to answer complex questions by searching the web.

## What You'll Learn

1. **LangGraph Fundamentals**: Understanding nodes, edges, and state management
2. **Agent Architecture**: How agents think, act, and observe
3. **Tool Integration**: Connecting external APIs (Tavily Search) to your agent
4. **AWS Bedrock Integration**: Using Amazon Nova Lite for AI reasoning

## Step 1: Install Required Packages

We need several packages:
- `langgraph`: Framework for building agent workflows as graphs
- `langchain-aws`: Integration with AWS Bedrock models
- `langchain-community`: Community tools including Tavily search
- `tavily-python`: Search API client
- `boto3`: AWS SDK for Python

In [22]:
!pip install -q langgraph langchain-aws langchain-community tavily-python boto3

## Step 2: Configure AWS Bedrock and API Keys

### Important Security Note
Store your credentials in Google Colab secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add these secrets:
   - `awsid`: Your AWS Access Key ID
   - `awssecret`: Your AWS Secret Access Key
   - `tavily`: Your Tavily API key (get free at https://tavily.com)

### What is AWS Bedrock?
AWS Bedrock provides access to foundation models from various providers including Amazon's own models. We're using Amazon Nova Lite, a fast and cost-effective model optimized for reasoning tasks.

In [23]:
import os
import boto3
from langchain_aws import ChatBedrock
from langchain_core.tools import tool
import requests
from datetime import datetime
import json
from google.colab import userdata

# Configure your AWS credentials using Colab secrets
AWS_ACCESS_KEY_ID = userdata.get('awsid')  # Set this in Colab secrets
AWS_SECRET_ACCESS_KEY = userdata.get('awssecret')  # Set this in Colab secrets
AWS_REGION = "us-east-1"  # Change if needed

# Configure Tavily API key for search
os.environ["TAVILY_API_KEY"] = userdata.get('tavily')

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

# Set up the Bedrock model (using Amazon Nova Lite for cost-effectiveness)
llm = ChatBedrock(
    client=bedrock_runtime,
    model_id="amazon.nova-lite-v1:0",
    model_kwargs={
        "temperature": 0,  # Low temperature for consistent tool calling
        "max_tokens": 4096
    }
)

print("✓ AWS Bedrock client initialized")
print("✓ Tavily API key configured")

✓ AWS Bedrock client initialized
✓ Tavily API key configured


## Step 3: Import Required Libraries

Let's break down what each import does:

### Core LangGraph Components
- `StateGraph`: Creates the graph structure for our agent
- `END`: Special node indicating the workflow should terminate

### Type Hints and Utilities
- `TypedDict`, `Annotated`: For defining structured state
- `operator`: For defining how state updates are handled

### Message Types
- `HumanMessage`: Represents user input
- `AIMessage`: Represents model responses
- `SystemMessage`: Sets the behavior/personality of the agent
- `ToolMessage`: Contains results from tool executions

### LangChain Components
- `ChatBedrock`: Wrapper for AWS Bedrock chat models (already initialized above)
- `TavilySearchResults`: Tool for web searching

In [24]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, BaseMessage
from langchain_community.tools.tavily_search import TavilySearchResults

## Step 4: Initialize the Search Tool

### What is Tavily?
Tavily is a search API optimized for AI agents. Unlike traditional search engines, it returns concise, relevant information perfect for LLMs to process.

### Why `max_results=2`?
We limit results to keep the context window manageable and reduce API costs. Two results typically provide enough information for most queries.

In [25]:
# Initialize the search tool
tools = [TavilySearchResults(max_results=2)]

print(f"Tool initialized: {tools[0].name}")
print(f"Tool description: {tools[0].description}")

Tool initialized: tavily_search_results_json
Tool description: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


## Step 7: Initialize the Agent

### Amazon Nova Lite Model
- **Speed**: Optimized for fast responses
- **Cost**: Cost-effective foundation model
- **Capabilities**: Supports tool calling and reasoning
- **Temperature**: Set to 0 for consistent, deterministic outputs

### System Prompt Design
The system prompt is crucial - it defines:
- The agent's personality and tone
- How it should use tools
- When to search vs. when to answer directly
- Response formatting preferences

In [26]:
class AgentState(TypedDict):
    """State that gets passed between nodes in the graph."""
    messages: Annotated[list[BaseMessage], operator.add]

## Step 6: Build the Agent Class

### Agent Architecture Overview

Our agent follows the **ReAct** pattern (Reasoning + Acting):
1. **Reason**: The LLM thinks about what to do
2. **Act**: It calls a tool if needed
3. **Observe**: It sees the tool's result
4. **Repeat**: Until it has enough information to answer

### The Three Key Methods

#### 1. `call_model()`
- Gets the current conversation
- Adds system instructions
- Calls the LLM
- Returns the LLM's response (which may include tool calls)

#### 2. `take_action()`
- Executes any tools the LLM requested
- Handles **parallel tool calling** (multiple tools at once)
- Returns tool results as messages

#### 3. `exists_action()`
- Checks if the LLM wants to use tools
- Returns True → go to action node
- Returns False → we're done, return to user

### Understanding the Graph Structure

```
START → LLM → [Decision]
                ├─ Has tool calls? → ACTION → LLM (loop)
                └─ No tool calls? → END
```

In [27]:
class Agent:
    def __init__(self, model, tools, system_message: str):
        """
        Initialize the agent.

        Args:
            model: The LLM to use (AWS Bedrock in our case)
            tools: List of tools available to the agent
            system_message: Instructions that define the agent's behavior
        """
        self.system_message = system_message

        # Create the graph structure
        graph = StateGraph(AgentState)

        # Add nodes (the boxes in our flowchart)
        graph.add_node("llm", self.call_model)
        graph.add_node("action", self.take_action)

        # Add conditional edge from LLM
        # This is the decision point: "Should I use a tool or respond?"
        graph.add_conditional_edges(
            "llm",  # Starting from the LLM node
            self.exists_action,  # Use this function to decide
            {
                True: "action",  # If True, go to action node
                False: END  # If False, we're done
            }
        )

        # Add edge from action back to LLM
        # After using a tool, the agent needs to think about the results
        graph.add_edge("action", "llm")

        # Set the entry point (where we start)
        graph.set_entry_point("llm")

        # Compile the graph into a runnable object
        self.graph = graph.compile()

        # Store tools as a dictionary for easy lookup
        self.tools = {t.name: t for t in tools}

        # Bind tools to the model
        # This tells the model what tools are available
        self.model = model.bind_tools(tools)

    def call_model(self, state: AgentState):
        """
        Call the LLM with the current conversation history.

        This is where the agent "thinks" about what to do next.
        """
        messages = state['messages']

        # Add system message at the beginning
        messages_with_system = [SystemMessage(content=self.system_message)] + messages

        # Call the model
        print("\n🤔 Agent is thinking...")
        response = self.model.invoke(messages_with_system)

        # Return as a state update (will be added to messages list)
        return {'messages': [response]}

    def take_action(self, state: AgentState):
        """
        Execute the tools that the LLM requested.

        Supports parallel tool calling - the model can request multiple
        tools at once for efficiency.
        """
        # Get the last message (which contains tool calls)
        last_message = state['messages'][-1]
        tool_calls = last_message.tool_calls

        results = []

        # Execute each tool call
        for tool_call in tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']

            print(f"\n🔧 Calling tool: {tool_name}")
            print(f"   Arguments: {tool_args}")

            # Find and call the tool
            tool = self.tools[tool_name]
            result = tool.invoke(tool_args)

            print(f"   Result preview: {str(result)[:100]}...")

            # Create a ToolMessage with the result
            results.append(
                ToolMessage(
                    content=str(result),
                    tool_call_id=tool_call['id']
                )
            )

        print("\n↩️  Going back to the model with results...")
        return {'messages': results}

    def exists_action(self, state: AgentState):
        """
        Check if the last message contains any tool calls.

        This is the decision function for our conditional edge.
        """
        last_message = state['messages'][-1]
        return len(last_message.tool_calls) > 0

# The AWS Bedrock model is already initialized above as 'llm'
# We'll use it directly in the agent

# Define the agent's behavior
system_message = """You are a helpful AI assistant with access to web search.

When answering questions:
1. Use search when you need current information (weather, news, recent events)
2. You can make multiple searches if needed to fully answer the question
3. Synthesize information from search results into clear, concise answers
4. If you can answer without searching (general knowledge), do so directly
5. Always cite your sources when using search results

Be conversational and helpful!"""

# Create the agent using the AWS Bedrock model we initialized earlier
agent = Agent(llm, tools, system_message)

print("✓ Using AWS Bedrock model (Amazon Nova Lite)")
print("✓ System message defined")
print("✓ Agent initialized and ready!")

## Step 8: Visualize the Agent's Graph

### Understanding the Visualization
- **Rectangles**: Nodes (actions the agent can take)
- **Diamonds**: Conditional decisions
- **Arrows**: Flow of execution
- **Loops**: The agent can cycle through thinking and acting

This visual representation helps us understand exactly how our agent will process requests.

In [28]:
from IPython.display import Image, display

try:
    # Generate and display the graph visualization
    display(Image(Agent.graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not generate graph visualization: {e}")
    print("This is optional - the agent will still work!")

Could not generate graph visualization: type object 'Agent' has no attribute 'graph'
This is optional - the agent will still work!


## Step 9: Test the Agent - Simple Query

### What to Expect
For weather queries, the agent will:
1. Recognize it needs current information
2. Call the Tavily search tool
3. Process the search results
4. Formulate a natural language response

Watch the execution flow in the output below!

In [33]:
# Helper function to run queries
# Create the agent using the AWS Bedrock model we initialized earlier
agent = Agent(llm, tools, system_message)

def ask_agent(question: str):
    """Ask the agent a question and return the response."""
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print(f"{'='*60}")

    # Create input in the format the agent expects
    input_messages = {'messages': [HumanMessage(content=question)]}

    # Run the agent
    result = agent.graph.invoke(input_messages)

    # Extract the final response
    final_message = result['messages'][-1]

    print(f"\n{'='*60}")
    print("Final Answer:")
    print(f"{'='*60}")
    print(final_message.content)

    return result

# Test 1: Simple weather query
result1 = ask_agent("What is the weather in Hong Kong?")

NameError: name 'system_message' is not defined

## Step 10: Test Parallel Tool Calling

### What is Parallel Tool Calling?
When the agent needs information about multiple independent things, modern LLMs can request multiple tools **at the same time** rather than sequentially.

### Why is This Useful?
- **Faster**: No waiting for one search to complete before starting another
- **Efficient**: Fewer round trips to the LLM
- **Natural**: Mimics how humans gather information

Watch how the agent calls search twice **before** going back to think!

In [34]:
# Test 2: Parallel tool calling (two independent searches)
result2 = ask_agent("What is the weather in Hong Kong and Los Angeles?")

NameError: name 'ask_agent' is not defined

## Step 11: Test Multi-Step Reasoning

### Sequential vs Parallel Tool Calling

This query requires **sequential** reasoning:
1. First search: "Who won the Super Bowl in 2024?"
2. Process the result (Kansas City Chiefs)
3. Second search: "GDP of Missouri" (where the Chiefs are based)

The agent can't do step 3 without the answer from step 1!

### Observing the Reasoning Process
Notice how the agent:
- Makes the first search
- Goes **back to the LLM** to think
- Then makes the second search
- Finally synthesizes both pieces of information

This demonstrates true **reasoning** capability, not just tool execution.

In [35]:
# Test 3: Multi-step reasoning (sequential tool calls)
result3 = ask_agent(
    "Who won the Nobel Prize in Physics 2024? What is the GDP of the state where that person is based?"
)

NameError: name 'ask_agent' is not defined

## Step 12: Inspect the Full Conversation History

### Understanding Message Types

The agent state contains all messages in order:
- **HumanMessage**: Your questions
- **AIMessage**: The LLM's responses (may include tool_calls)
- **ToolMessage**: Results returned by tools

### Why This Matters
This complete history enables:
- **Multi-turn conversations**: The agent remembers context
- **Debugging**: See exactly what happened at each step
- **Learning**: Understand how the agent reasoned
- **Persistence**: Could save and resume conversations

In [36]:
# Let's examine the full message history from the last query
print("\n" + "="*60)
print("COMPLETE MESSAGE HISTORY")
print("="*60)

for i, msg in enumerate(result3['messages'], 1):
    print(f"\n{i}. {msg.__class__.__name__}:")
    print("-" * 60)

    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print("Tool Calls:")
        for tc in msg.tool_calls:
            print(f"  - {tc['name']}({tc['args']})")

    if msg.content:
        content_preview = msg.content[:200] + "..." if len(msg.content) > 200 else msg.content
        print(f"Content: {content_preview}")


COMPLETE MESSAGE HISTORY


NameError: name 'result3' is not defined

## Step 13: Try Your Own Questions!

### Experiment Ideas

Try questions that require:
1. **No search**: "What is 2+2?" or "Explain photosynthesis"
2. **Single search**: "What's the current stock price of Apple?"
3. **Parallel searches**: "Compare the populations of Tokyo and New York"
4. **Sequential reasoning**: "Who is the CEO of Tesla? What other companies do they run?"
5. **Complex analysis**: "What are the top 3 news stories today and how are they related?"

### Understanding Limitations
The agent can:
- ✅ Search for current information
- ✅ Reason across multiple searches
- ✅ Synthesize information

The agent cannot:
- ❌ Remember previous conversations (each query is independent)
- ❌ Access information not available via search
- ❌ Perform actions (only read information)

In [39]:
# Try your own question!
my_question = "What are the latest developments in AI this week?"

result = ask_agent(my_question)

NameError: name 'ask_agent' is not defined

## Key Concepts Summary

### 1. **LangGraph State Management**
- State flows through nodes
- `operator.add` accumulates messages
- Complete history enables reasoning

### 2. **Agent Architecture (ReAct Pattern)**
```
Thought → Action → Observation → Thought → ...
```
- **Thought**: LLM decides what to do
- **Action**: Execute tools
- **Observation**: Process results
- **Loop**: Until task is complete

### 3. **Conditional Logic**
- Edges can be conditional (decision points)
- Enables dynamic workflows
- Agent chooses its own path

### 4. **Tool Integration**
- Tools extend agent capabilities
- LLM decides when to use them
- Results feed back into reasoning

### 5. **Parallel vs Sequential**
- **Parallel**: Independent tasks done simultaneously
- **Sequential**: Each step depends on previous results
- LLM automatically chooses the right strategy

## Next Steps

To extend this agent, you could:
1. **Add more tools**: Calculator, database access, API calls
2. **Add memory**: Store conversation history across sessions
3. **Add guardrails**: Validate tool inputs/outputs
4. **Add human-in-the-loop**: Require approval for certain actions
5. **Multi-agent systems**: Have multiple agents collaborate

## Resources

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Tavily Search API](https://tavily.com/)
- [LangChain Hub](https://smith.langchain.com/hub)